In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# Any results you write to the current directory are saved as output.

from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from catboost import CatBoostRegressor
from sklearn.decomposition import PCA

from google.colab import files

In [0]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

In [0]:
train = pd.read_csv("train (1).csv")
test = pd.read_csv("test (1).csv")
test_id=test.id

#train = train.drop(['id'], axis=1)
#train = train.drop_duplicates()

#train = train.reset_index(drop=True)

In [0]:
train.head()

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev,vector_pca0
0,1,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.0680,3.4387,2.303089
1,2,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.2490,2.9210,-10.413997
2,3,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.1821,2.7438,-1.223099
3,4,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,0.2172,3.3492,-4.410966
4,5,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,0.0505,1.3793,-10.700240


In [0]:
vector = np.vstack((train[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values,
                    test[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values))

In [0]:
pca = PCA().fit(vector)
train['vector_pca0'] = pca.transform(train[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]
test['vector_pca0'] = pca.transform(test[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]


X = train.drop(['id','bandgap_energy_ev','formation_energy_ev_natom'], axis=1)
Y_feen = np.log(train['formation_energy_ev_natom']+1)
Y_bee = np.log(train['bandgap_energy_ev']+1)

test = test.drop(['id'], axis = 1)

In [0]:
def runCatBoost(x_train, y_train,x_test, y_test,test,depth):
    model=CatBoostRegressor(iterations=1200,
                            learning_rate=0.03,
                            depth=depth,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=99,
                            od_type='Iter',
                            od_wait=50)
    model.fit(x_train, y_train, eval_set=(x_test, y_test), use_best_model=True, verbose=False)
    y_pred_train=model.predict(x_test)
    rmsle_result = rmsle(np.exp(y_pred_train)-1,np.exp(y_test)-1)
    y_pred_test=model.predict(test)
    return y_pred_train,rmsle_result,y_pred_test

In [0]:
pred_full_test_cat_feen = 0
mse_cat_list_feen=[]
kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=30)
for dev_index, val_index in kf.split(X):
    dev_X, val_X = X.loc[dev_index], X.loc[val_index]
    dev_y, val_y = Y_feen.loc[dev_index], Y_feen.loc[val_index]
    y_pred_feen,rmsle_feen,y_pred_test_feen=runCatBoost(dev_X, dev_y, val_X, val_y,test,depth=4)
    mse_cat_list_feen.append(rmsle_feen)
    pred_full_test_cat_feen = pred_full_test_cat_feen + y_pred_test_feen
mse_cat_feen_mean=np.mean(mse_cat_list_feen)
print("Mean cv score : ", np.mean(mse_cat_feen_mean))
y_pred_test_feen=pred_full_test_cat_feen/10

Mean cv score :  0.032157606465501275


In [0]:
pred_full_test_cat_bee = 0
mse_cat_list_bee=[]
kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=30)
for dev_index, val_index in kf.split(X):
    dev_X, val_X = X.loc[dev_index], X.loc[val_index]
    dev_y, val_y = Y_bee.loc[dev_index], Y_bee.loc[val_index]
    y_pred_bee,rmsle_bee,y_pred_test_bee=runCatBoost(dev_X, dev_y, val_X, val_y,test,depth=4)
    mse_cat_list_bee.append(rmsle_bee)
    pred_full_test_cat_bee = pred_full_test_cat_bee + y_pred_test_bee
mse_cat_bee_mean=np.mean(mse_cat_list_bee)
print("Mean cv score : ", np.mean(mse_cat_bee_mean))
y_pred_test_bee=pred_full_test_cat_bee/10

Mean cv score :  0.08645899596386686


In [0]:
sub=pd.DataFrame()
sub["id"]=test_id
sub["formation_energy_ev_natom"]=np.clip(np.exp(y_pred_test_feen)-1, 0, None)
sub["bandgap_energy_ev"]=np.clip(np.exp(y_pred_test_bee)-1, 0, None)
rmsle_total=np.mean([mse_cat_bee_mean,mse_cat_feen_mean])
print(rmsle_total)
sub.to_csv("Nomad2018.csv",index=False)

0.05930830121468407


In [0]:
sub.to_csv("Nomad2018.csv",index=False)
files.download("Nomad2018.csv")